In [1]:
import import_ipynb 
import numpy as np
import os
import xmltodict
import xml.dom.minidom
import xml.etree.cElementTree as ET
from copy import copy, deepcopy
from PIL import Image
from datetime import datetime
import time
from generateFolderStructur import GenerateFolderStructur 
import matplotlib.colors as c

importing Jupyter notebook from generateFolderStructur.ipynb


In [17]:
class Place():
    
    """
    Eingangsdaten:
    Zugeschnittene Bilder, xml Ordner, segmentierte Bilder, Bildgrösse, Channels, iou threshold
    
    Konstruktor genieriert ein dictonary, welches wie folgt aussieht:
    
    {Pfad_Klassen:{Klassenname_1:{0:Bildname_1, 1:Bildname1}}
                 :{Klassenname_2:{0:Bildname_1, 1:Bildname2}}
    }
    
    2 Varianten wie die einzelen Zeichen platziert werden können:
    
    1. rdm platziert
    2. ....
    
    """
    
    def __init__(self, path, xml_path, segemented_image_path, size, depth, iou_threshold):
        self._path = path
        self._xml_path = xml_path
        self._segemented_image_path = segemented_image_path
        self._size = size
        self._depth= depth
        self._iou_threshold = iou_threshold
        self._dictonary = {'path':{'folder_name':{0:'image_name'}}}
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._bounding_boxes = np.array([0,0,0,0])
        self._body_name = ''
        self._date_time = ''
        self._placed_image_path = ''
        self._x_rdm_min = 0
        self._y_rdm_min = 0
        self._x_rdm_max = 0
        self._y_rdm_max = 0
        self._dict_folders = ''
        self._number_dict_folders = 0
        self._number_dict_images = 0
        self._rdm_image = 0
        self._filename = ''
        self._place_trials = 0
        self._result = False
        self._finish = False
        self._placed = False
        self._modified_dirs_3 = []
        self._change_folders = []
        self._part = []
        
        path_1, dirs_1, files_1 = next(os.walk(self._path))
        self._dictonary = {path_1:{}}
        for folder in dirs_1:
            self._dictonary[path_1][folder] = {}
#             print(folder)
            path_2, dirs_2, files_2 = next(os.walk(path_1 + folder))
            for i, files in zip(range(len(files_2)), files_2):
                self._dictonary[path_1][folder][i] = {files}
#                 print(files) 

    def get_head(self):
#         print('get_head')self._placed_image_path
        now = datetime.now()
        self._date_time = now.strftime("%Y%m%d_%H%M%S")
        slice_position_r = self._segemented_image_path.rfind('/')
        slice_position_l = self._segemented_image_path[:slice_position_r].rfind('/')
        self._filename = self._date_time + self._filename
        folder = self._segemented_image_path[slice_position_l+1:slice_position_r]
#         path = self._segemented_image_path + folder + '/' + self._filename +  '.bmp'
        path = self._segemented_image_path + self._filename +  '.bmp'        
        string_head =   '<annotation>'\
                        '<folder>'+str(folder)+'</folder>'\
                        '<filename>'+str(self._filename)+'.bmp'+'</filename>'\
                        '<path>'+path+'</path>'\
                        '<source>'\
                        '<database>Unknown</database>'\
                        '</source>'\
                        '<size>'\
                            '<width>'+str(self._size)+'</width>'\
                            '<height>'+str(self._size)+'</height>'\
                            '<depth>'+str(self._depth)+'</depth>'\
                        '</size>'\
                        '<segmented>0</segmented>'\
                        '</annotation>'
        return string_head

    
    def get_body(self,image_name,xmin,ymin,xmax,ymax):
#         print('get_body')
        tmp_slice_r = self._placed_image_path.rfind('_tailored')
        tmp_slice_l = self._placed_image_path[:tmp_slice_r].rfind('_')
        self._filename = self._filename + '_' + self._placed_image_path[tmp_slice_l+1:tmp_slice_r]
        
        tmp_start = image_name.find('TailoredImages/')
        tmp_length = len('TailoredImages/')
        tmp_end = image_name.rfind('/')
        name = image_name[tmp_start+tmp_length:tmp_end]
        
        string_object = '<object>'\
                        '<name>'+str(name)+'</name>'\
                        '<pose>Unspecified</pose>'\
                        '<truncated>0</truncated>'\
                        '<difficult>0</difficult>'\
                        '<bndbox>'\
                            '<xmin>'+str(xmin)+'</xmin>'\
                            '<ymin>'+str(ymin)+'</ymin>'\
                            '<xmax>'+str(xmax)+'</xmax>'\
                            '<ymax>'+str(ymax)+'</ymax>'\
                            '</bndbox>'\
                        '</object>'
        return string_object

    
    def iou_check(self):
#         print('iou_check')
        if np.count_nonzero(self._segmented_image) == 0:
            self._bounding_boxes  = np.array([0,0,0,0])
            self._bounding_boxes[0] = self._x_rdm_min
            self._bounding_boxes[1] = self._y_rdm_min
            self._bounding_boxes[2] = self._x_rdm_max
            self._bounding_boxes[3] = self._y_rdm_max
#             print('no iou check')
            self._result = True
        else:
            bounding_box = np.array([0,0,0,0])
            bounding_box[0] = self._x_rdm_min
            bounding_box[1] = self._y_rdm_min
            bounding_box[2] = self._x_rdm_max
            bounding_box[3] = self._y_rdm_max
            length = int(len(self._bounding_boxes)/4)
#             print('place_trials: ', self._place_trials)
            tmp_iou = 0
            tmp_max_iou = 0
            
            for i in range(length):
                tmp_start = 0+(4*i)
                tmp_end = 3+(4*i)
                
#                 print(tmp_start,tmp_end)
                tmp_bounding_boxes = self._bounding_boxes[tmp_start:tmp_end+1]
                tmp_iou = self.bb_intersection_over_union(bounding_box, tmp_bounding_boxes)

                if tmp_iou >= tmp_max_iou:
                    tmp_max_iou = tmp_iou
                    
            
            if tmp_max_iou <= self._iou_threshold: 
                self._bounding_boxes = np.append(self._bounding_boxes,bounding_box)
#                 print('bounding: ',self._bounding_boxes)
                self._place_trials = 0
                self._result = True
            else:
                self._result = False
                if self._place_trials == 3:
                    self.create_xml()
                    self.rdm()
                    
                else:
                    self._place_trials += 1
                    self.place()         
    
    
    def place(self):
#         print('place')
        with Image.open(self._placed_image_path) as image:
            self._img = np.array(image)
            y,x,z = self._img.shape
            if x >= self._size-10 or y >= self._size-10:
                print('Folgendes Bild ist grösser als: ', str(self._size-10))
                print(self._placed_image_path)
            else:
                x_max = self._size - x - 5
                y_max = self._size - y - 5
                self._x_rdm_min = np.random.randint(5,x_max)
                self._y_rdm_min = np.random.randint(5,y_max)
                self._x_rdm_max = self._x_rdm_min + x
                self._y_rdm_max = self._y_rdm_min + y
                ### _result wird True gesetzt, wenn IOU kleiner als _iou_threshold ist, dies wird in der iou_check verifiziert. ###
                
                self.iou_check()

                y,x,z = (self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:]).shape

                if  self._result == True:
                    self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] = self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] + self._img
                    self._placed = True
                    self._result = False

        
        
    def create_xml_segmented_image(self, string):
#         print('create_xml_segmented_image')
        dom = xml.dom.minidom.parseString(string)
        pretty_xml_as_string_body = dom.toprettyxml()
        pretty_xml_as_string_body_without_version = pretty_xml_as_string_body[23:]
        filepath = self._xml_path + '/' + self._filename + '.xml'
        with open(filepath, 'w+') as file:
            file.write(pretty_xml_as_string_body_without_version)
            
        img_uint8 = Image.fromarray(np.uint8(self._segmented_image))
        img_uint8.save(self._segemented_image_path +'/'+self._filename+'.bmp')
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._filename = ''
            
        
    def create_xml_body(self,):
#         print('create_xml_body')
        slice_position = self._placed_image_path.find('_')
        object_name = self._placed_image_path[:slice_position]
        xml_name_tmp = self.get_body(self._placed_image_path,self._x_rdm_min-1,self._y_rdm_min-1,self._x_rdm_max+1,self._y_rdm_max+1)
        self._body_name = self._body_name + xml_name_tmp
#         print(self._body_name)
              
    def create_xml(self):
#         name = str(Place.placeIt_counter) + 
#         print(xml_head)
        if  self._place_trials == 3 or len(list(self._dictonary[self._path])) == 0:
            xml_head = self.get_head()
            slice_position = xml_head.find('</annotation>')
            complete_xml = xml_head[:slice_position] + self._body_name + xml_head[slice_position:]
    #         print(complete_xml)
            self.create_xml_segmented_image(complete_xml)
            self._body_name = ''  
            self._place_trials = 0
        else:
            self.rdm()
    
    def delete_dict(self): 
#         print('delete')
        del self._dictonary[self._path][self._dict_folders][self._rdm_image]
        if len(list(self._dictonary[self._path][self._dict_folders]))== 0:
            del self._dictonary[self._path][self._dict_folders]
        if len(list(self._dictonary[self._path])) == 0:
            self._finish =  True
        else:
            self._finish =  False
    
    
    def rdm(self):

#         print('rdm')
        ### Zufälliger Ordner von Zeichen z.B. A oder B ###
        self._dict_folders = list(self._dictonary[self._path].keys())
        self._number_dict_folders = len(self._dict_folders)
        
#         print(self._dictonary)
        
        if self._number_dict_folders != 0:
            rdm_folder = np.random.randint(self._number_dict_folders)
            self._dict_folders = self._dict_folders[rdm_folder]
            
            ### Zufälliges Bild von oben bestimmten Zeichen ###
            dict_images = list(self._dictonary[self._path][self._dict_folders].keys())
            
            self._number_dict_images = len(dict_images)
            rdm_element = np.random.randint(self._number_dict_images)
            self._rdm_image = dict_images[rdm_element]
            dict_image_name = str(self._dictonary[self._path][self._dict_folders][self._rdm_image])

            r_tmp = dict_image_name.find('}')
            dict_image_name = dict_image_name[2:r_tmp-1]


            ### Bild Pfad des Zeichens ###
            self._placed_image_path = self._path + self._dict_folders + '/' +  dict_image_name
            self.place()

        if self._placed == True:
            self.create_xml_body() 
            if len(list(self._dictonary[self._path])) != 0:
                self.delete_dict()
                self.create_xml()
                
        
    
    def bb_intersection_over_union(self, boxA, boxB):
#         print('bb_intersection_over_union')
        
        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)

#         print('iou: ', iou)
        return iou

    ### hier ist der Fehler irgendwo #####
    def get_change_folders(self, characters):
        if len(characters)==1:
#             print('GenerateFolderStructur.dictSigns.keys(): ', GenerateFolderStructur.dictSigns.keys()) 
        
            if characters in GenerateFolderStructur.dictSigns.keys(): # GenerateFolderStructur.dictSigns.keys() ist ['-', '.', '/', ':']
                self._change_folders.append(GenerateFolderStructur.dictSigns[characters])
            else:
                self._change_folders.append(characters)
                print(characters)
                
        elif characters.find('-')!= -1:
            
            if GenerateFolderStructur.all_characters.find(characters[0]) != -1:
                tmp_start = GenerateFolderStructur.all_characters.find(characters[0])
                tmp_end = GenerateFolderStructur.all_characters.find(characters[2])
                for i in range(tmp_start,tmp_end+1):
                    self._change_folders.append(GenerateFolderStructur.all_characters[i])
            else:
                index = characters.find('-')
                tmp_start = int(characters[:index])
                tmp_end = int(characters[index+1:])
                for j in range(tmp_start,tmp_end+1):
                    self._change_folders.append(j)
                    
        elif len(characters)>1:
            self._change_folders.append(characters)
#         print('_resize_folder: ',self._change_folders)      
    
    def get_parts(self,tmp_part):
        file_length = self._number_characters
        index = tmp_part.find('/')
        tmp_start = int(tmp_part[:index])
        tmp_end = int(tmp_part[index+1:])

        self._part = []
        self._part.append(str(tmp_start-1))
        for i in range(int(file_length/tmp_end)):
            if int(self._part[i])+tmp_end < file_length: 
                self._part.append(str(int(self._part[i])+tmp_end))        
#         print('part: ',self._part)
    
    def resize_image(self,*args):
        tmp_size = args[0]
        y_min, y_max, x_min, x_max = tmp_size[0],tmp_size[1],tmp_size[2],tmp_size[3]
        image_path = args[1]
        with Image.open(image_path) as image:
            width, height = image.size
            if y_min == y_max:
                y_max = y_min+1
            if x_min == x_max:
                x_max = x_min+1
            rdm_width = np.random.randint(x_min,x_max)
            rdm_height = np.random.randint(y_min, y_max)
            image_resized = image.resize((rdm_width,rdm_height)) 
            image_resized.save(image_path)
    
    def get_images(self,f_parameter,*args):
        for i in range(len(self._change_folders)):
            for j in range(len(self._modified_dirs_3)):
                if self._modified_dirs_3[j].find(str(self._change_folders[i])) !=-1:
                    path_5, dirs_5, files_5 = next(os.walk(self._path+self._dirs_3[j]+'/'))
                    for k in range(len(self._part)):
                        image_path = path_5 + files_5[int(self._part[k])]
                        f_parameter(args,image_path)
                        
    
    def get_modified_dirs_3(self):
        if len(self._modified_dirs_3) == 0:
            path_3, self._dirs_3, files_3 = next(os.walk(self._path))
            
            for j in range(len(self._dirs_3)):
                path_4, dirs_4, files_4 = next(os.walk(self._path+self._dirs_3[j]+'/'))
                if j == 0:
                    self._number_characters = len(files_4)
                else:
                    if self._number_characters != len(files_4):
                        print('Nicht gleich viele Zeichen vorhanden!!!')
                        print('Im Folgende Ordner hat es eine unterschiedliche Anzahl Zeichen: ', self._dirs_3[j])
                
            
            for i in range(len(self._dirs_3)):
                if self._dirs_3[i].find('_')!=-1:
                    self._modified_dirs_3.append(self._dirs_3[i][0])
                else:
                    self._modified_dirs_3.append(self._dirs_3[i])
#         print(self._modified_dirs_3)    
    
    def get_modified_dirs_3_change_folder_parts(self,characters,tmp_part):
        self._change_folders = [] 
        self._part = []
        self.get_change_folders(characters)
        self.get_modified_dirs_3()
        self.get_parts(tmp_part)
        
    
    def resize(self, characters, tmp_part, y_min, y_max, x_min, x_max):
        
        ### Ermittelt modified_dirs_3 change_folder and parts ###
        self.get_modified_dirs_3_change_folder_parts(characters, tmp_part)
        
        #######################################################
        self.get_images(self.resize_image, y_min, y_max, x_min, x_max)
        
    
    def color_hsv(self, characters, tmp_part, h_mean, s_mean, v_mean, h_std, s_std, v_std):

        ### Ermittelt modified_dirs_3 change_folder and parts ###
        self.get_modified_dirs_3_change_folder_parts(characters, tmp_part)
        
        #######################################################
        self.get_images(self.color_image_hsv, h_mean, s_mean, v_mean, h_std, s_std, v_std)
        
    def color_image_hsv(self,*args):
        """
        Eingangsparameter:
        Klasse die verändert werden soll: '1'   'a-z' oder 'hallo'
        Wieviele dieser Klasse verändert werden sollen: '1/1'   '1/4' oder '3/5' 
        '3/5' bedeutet, dass immer auf 5 gezählt wird und dabei jeder 3 veränder wird.
        H: 234   1-360
        S: 55    1-100
        V: 70    1-100
        H std: 5
        S std: 3
        V std: 4
        
        """
        tmp_hsv = args[0]
        h_mean, s_mean, v_mean = tmp_hsv[0], tmp_hsv[1], tmp_hsv[2]
        h_std, s_std, v_std = tmp_hsv[3], tmp_hsv[4], tmp_hsv[5]
        image_path = args[1]
        
        with Image.open(image_path) as image:

            width, height = image.size
            hsv_matrix = np.zeros((height,width,3))
            h_vector = np.random.normal(h_mean,h_std,(height,width))
            s_vector = np.random.normal(s_mean,h_std,(height,width))
            v_vector = np.random.normal(v_mean,h_std,(height,width))
            h_vector[h_vector>360] = 360
            h_vector[h_vector<1] = 1
            s_vector[s_vector>100] = 100
            s_vector[s_vector<1] = 1
            v_vector[v_vector>100] = 100
            v_vector[v_vector<1] = 1
            h_vector = h_vector/360
            s_vector = s_vector/100
            v_vector = v_vector/100
            hsv_matrix[:,:,0] = h_vector
            hsv_matrix[:,:,1] = s_vector
            hsv_matrix[:,:,2] = v_vector
            hsv_matrix = c.hsv_to_rgb(hsv_matrix)*255
            hsv_matrix = hsv_matrix.astype(int)
#             print(hsv_matrix)
            
            tmp_matrix = np.array(image)
            tmp_matrix[tmp_matrix!=0] = 1
            color_image = np.multiply(tmp_matrix,hsv_matrix)
            color_image_uint8 = Image.fromarray(np.uint8(color_image))       
            color_image_uint8.save(image_path)

        ### colorsys ###
        ### https://docs.python.org/2/library/colorsys.html ###

In [18]:
obj33 = Place('C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/','C:/Users/User/Desktop/Test/GeneratedData/XML_Files/','C:/Users/User/Desktop/Test/GeneratedData/SegmentedImages',512,3,0.08)

In [19]:
# print(GenerateFolderStructur.dictSigns)
# obj33.resize('a','1/1',100,101,40,41)
obj33.color_hsv('a','1/1',0,0,100,0,0,0) # weiss

a


In [16]:
obj33.color_hsv('a','1/1',6,100,100,0,0,0) # rot

In [8]:
# print(obj33._dictonary)

In [9]:
# obj33.rdm()

In [10]:
# path, dirs, files = next(os.walk('C:/Users/User/Desktop/Test/GeneratedData/XML_Files'))
# print(files[6])
# with open(path +'/' + files[6],'r') as file:
#     text = []
#     for line in file:
#         text.append(line.rstrip())
        
# print(text)

In [11]:
### Öffnet das file obj.names und macht zwei dictionarys. ###
# dict_id_class:   key: id,    value: class
# dict_class_id:   key: class, value: id

# with open('C:/Users/User/Desktop/Test/GeneratedData/data/' + 'obj.names','r+') as names:
#     dict_id_class = {}
#     dict_class_id = {}
#     number_lines = 0
#     for line in names:
#         index = line.find('\n')
#         dict_class_id[str(line[:index])] = number_lines
#         dict_id_class[number_lines] = str(str(line[:index]))
#         number_lines += 1
# #     print(dict_class_id)
# #     print(dict_id_class)

############################################################################
### Diese Funktion sollte erste nach der zusammenführung von
### background und segmentedImages stattfinden ###


### Liest jedes xml file aus und macht daraus ein "filename + .txt" file. Je Zeile info zu bounding boxes.
# <id> <x> <Y> <w> <h> // alles relativ zur Bildgrösse (Dividiert durch Breite,Höhe des Bildes [0-1.0]
# id: Klassen_index des obj.names file,   x,y: Mittelpunkt der bounding boxes   w,h: Breite,Höhe der bouding boxes

# path_xml, dirs_xml, files_xml = next(os.walk('C:/Users/User/Desktop/Test/GeneratedData/XML_Files'))

# for xml in files_xml:
#     with open(path_xml + '/' + xml) as fd:
#         data = fd.read() # string
#         doc = xmltodict.parse(data) # dictonary
#         dom = parseString(data) # für suche mit len

#         path = doc['annotation']['path']
#         width = int(doc['annotation']['size']['width'])
#         height = int(doc['annotation']['size']['height'])
#         filename = doc['annotation']['filename']
#         number_objects = len(dom.getElementsByTagName('object'))
        

#         tmp_index = filename.find('.')   
#         with open('C:/Users/User/Desktop/Test/GeneratedData/data/' + filename[:tmp_index] + '.txt' ,'w+') as image_txt: # image.txt
        
#             for i in range(number_objects):
#                 name = doc['annotation']['object'][i]['name']
#                 xmin = int(doc['annotation']['object'][i]['bndbox']['xmin'])
#                 ymin = int(doc['annotation']['object'][i]['bndbox']['ymin'])
#                 xmax = int(doc['annotation']['object'][i]['bndbox']['xmax'])
#                 ymax = int(doc['annotation']['object'][i]['bndbox']['ymax'])
#                 x = (xmax + xmin) / (2.0*width)
#                 y = (ymax + ymin) / (2.0*height)
#                 w = (xmax - xmin) / width
#                 h = (ymax - ymin) / height
                
#                 image_txt.write(str(dict_class_id[name]) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')

# path_data, dirs_data, files_data = next(os.walk('C:/Users/User/Desktop/Test/GeneratedData/data'))

